In [ ]:
from Preference_Embeddings.Embeddings import PreferenceEmbedding, FuncDataset, evaluate
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import LambdaLR

In [10]:
# 6) Model, optimizer, loss
model = PreferenceEmbedding(in_dim=2)
func1 = lambda x: x.pow(2).sum()
func2 = lambda x: 100 * torch.abs(torch.sqrt(x[1] - 0.01*x[0])) + 0.01 * torch.abs(x[0] + 10)

In [11]:
batch_size = 64
lr         = 1e-1
epochs     = 500
# 5) DataLoader
dataset = FuncDataset(num_pairs=50_000, func = func1, minimize=True, in_dim=2)
loader  = DataLoader(dataset, batch_size=512, shuffle=True)
total_steps = epochs * len(loader)
warmup_steps = int(0.1 * total_steps)

#optimizer = torch.optim.LBFGS(model.parameters(), lr=lr, max_iter=50)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
def lr_lambda(step):
    if step < warmup_steps:
        return float(step) / float(max(1, warmup_steps))
    else:
        # cosine decay after warmup
        progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        return 0.5 * (1.0 + torch.cos(torch.tensor(torch.pi * progress)))
scheduler = LambdaLR(optimizer, lr_lambda)
loss_fn = nn.BCELoss()
def closure():
    optimizer.zero_grad()
    preds = model(X, Xp)
    loss  = loss_fn(preds, Y)
    loss.backward()
    return loss


In [15]:
patience = 10
threshold = 1e-10
best_loss = torch.inf
bad_steps = 10
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = LambdaLR(optimizer, lr_lambda)
for epoch in range(1, epochs):
    # pull the entire dataset in one batch
    X, Xp, Y = next(iter(loader))
    #X, Xp, Y = X.to(device), Xp.to(device), Y.to(device)

    optimizer.zero_grad()
    preds = model(X, Xp)
    loss  = loss_fn(preds, Y)
    loss.backward()
    optimizer.step()
    scheduler.step()

    # recompute for logging
    with torch.no_grad():
        loss_val = loss_fn(model(X, Xp), Y).item()
        if loss_val < best_loss - threshold:
            best_loss = loss_val
            bad_steps = 0
        else:
            bad_steps += 1
    if bad_steps >= patience:
        pass
    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} — loss: {loss_val:.6f}")

Epoch 010 — loss: 0.018386
Epoch 020 — loss: 0.020493
Epoch 030 — loss: 0.011020
Epoch 040 — loss: 0.010171
Epoch 050 — loss: 0.008826
Epoch 060 — loss: 0.014865
Epoch 070 — loss: 0.015429
Epoch 080 — loss: 0.025624
Epoch 090 — loss: 0.013852
Epoch 100 — loss: 0.021805
Epoch 110 — loss: 0.008451
Epoch 120 — loss: 0.038114
Epoch 130 — loss: 0.021774
Epoch 140 — loss: 0.011073
Epoch 150 — loss: 0.028083
Epoch 160 — loss: 0.015589
Epoch 170 — loss: 0.011158
Epoch 180 — loss: 0.018997
Epoch 190 — loss: 0.009813
Epoch 200 — loss: 0.012800
Epoch 210 — loss: 0.017569
Epoch 220 — loss: 0.011136
Epoch 230 — loss: 0.010027
Epoch 240 — loss: 0.014313
Epoch 250 — loss: 0.008176
Epoch 260 — loss: 0.022520
Epoch 270 — loss: 0.009340
Epoch 280 — loss: 0.014176
Epoch 290 — loss: 0.015348
Epoch 300 — loss: 0.018964
Epoch 310 — loss: 0.008947
Epoch 320 — loss: 0.021490
Epoch 330 — loss: 0.014179
Epoch 340 — loss: 0.007700
Epoch 350 — loss: 0.009031
Epoch 360 — loss: 0.016072
Epoch 370 — loss: 0.007321
E

In [16]:
res = evaluate(model, loader)

Accuracy : 0.9966
Precision: 0.9965
Recall   : 0.9968
F1-score : 0.9966
ROC-AUC  : 0.9999
Confusion Matrix:
[[24901    87]
 [   81 24931]]


In [17]:
torch.save(model.state_dict(), '../Embedding_Model_Weights/model_x_squared.pt')

In [68]:
loss_val

0.2851998209953308